In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

vec = cPickle.load(open('../data/vectorizers/allfields_with_embedding_5000.p', 'rb'))
cohen_vec = cPickle.load(open('../data/vectorizers/cohendata_dedup_5000.p'))

Using TensorFlow backend.


[]


In [2]:
index = vec.index['abstract']
vec.X = vec.X[index[0]:index[1]]

In [4]:
import numpy as np
train_X = vec.X
X_tf = np.zeros((train_X.shape[0], vec.vocab_size))
for i in range(len(train_X)) :
    X_tf[i, train_X[i, :]] = 1.

X_tf = X_tf[:, 2:]
train_Xtf = X_tf

In [5]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras import optimizers

learning_rate = 5e-5
batch_size = 64
vocab_size = vec.vocab_size - 2
intermediate_dim = 500
latent_dim = 200
epochs = 1000
epsilon_std = 1.0
activation = 'tanh'

x = Input(shape=(vocab_size,), name='x')
h = Dense(intermediate_dim, activation=activation, name='h')(x)
mu = Dense(latent_dim, name='mu')(h)
log_sigma2 = Dense(latent_dim, name='l')(h)
encoder = Model(x, mu)

# reparameterized sampler for normal distributions
def sample_norm(args):
    '''reparameterized sampling from normal distribution'''
    mu, log_var = args
    epsilon = K.random_normal(shape=(K.shape(mu)[0], latent_dim,), mean=0.)
    return mu + K.exp(0.5 * log_var) * epsilon

# decoder / generative network
z = Lambda(sample_norm, output_shape=(latent_dim,), name='z')([mu, log_sigma2])
e = Dense(vocab_size, name='e')(z)

def log_softmax(x, axis=None):
    x0 = x - K.max(x, axis=axis, keepdims=True)
    log_sum_exp_x0 = K.log(K.sum(K.exp(x0), axis=axis, keepdims=True))
    return x0 - log_sum_exp_x0

def kl_loss(x, e): 
    return (- 0.5 * K.sum(1 + log_sigma2 - K.square(mu) - K.exp(log_sigma2), axis=-1))


def cross_ent_loss(x, e): 
    return - K.sum(x * log_softmax(e, axis=-1), axis=-1) 
    

def vae_loss(x, e):
    xent_loss = cross_ent_loss(x, e)
    kld = kl_loss(x, e)
    return xent_loss + kld


opt = optimizers.adam(lr=learning_rate)
vae = Model(x, e)
vae.compile(optimizer=opt, 
            loss=vae_loss)

In [7]:
from keras import callbacks
patience = 3
earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='min')

vae.fit(train_Xtf,  
        train_Xtf, 
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1, 
        callbacks=[earlyStopping], 
        validation_split=0.1)

Train on 37605 samples, validate on 4179 samples
Epoch 1/1000
37605/37605 [==============================] - 7s - loss: 700.8528 - val_loss: 695.0157
Epoch 2/1000
37605/37605 [==============================] - 7s - loss: 700.5169 - val_loss: 694.8646
Epoch 3/1000
37605/37605 [==============================] - 7s - loss: 700.1008 - val_loss: 694.0322
Epoch 4/1000
37605/37605 [==============================] - 7s - loss: 699.7725 - val_loss: 694.0046
Epoch 5/1000
37605/37605 [==============================] - 7s - loss: 699.2780 - val_loss: 693.4880
Epoch 6/1000
37605/37605 [==============================] - 7s - loss: 698.9924 - val_loss: 693.3763
Epoch 7/1000
37605/37605 [==============================] - 7s - loss: 698.6227 - val_loss: 693.2508
Epoch 8/1000
37605/37605 [==============================] - 7s - loss: 698.1155 - val_loss: 692.7852
Epoch 9/1000
37605/37605 [==============================] - 7s - loss: 697.7956 - val_loss: 692.8701
Epoch 10/1000
37605/37605 [===============

In [9]:
X = cohen_vec.X
cohen_X_tf = np.zeros((X.shape[0], vec.vocab_size))
for i in range(len(X)) :
    cohen_X_tf[i, X[i, :]] = 1.

cohen_X_tf = cohen_X_tf[:, 2:]
cohen_X_tf.shape

(1767, 5000)

In [10]:
embedds = encoder.predict(cohen_X_tf)

In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [16]:
from sklearn.preprocessing import normalize
embedds_n = normalize(embedds, 'l2')
scores = np.dot(embedds_n, embedds_n.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [17]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.824494652055
{'NSAIDS_processed': 0.8878757023735081, 'ACEInhibitors_processed': 0.82898028932750711, 'SkeletalMuscleRelaxants_processed': 0.71413229709440951, 'Triptans_processed': 0.90845402091448257, 'OralHypoglycemics_processed': 0.87597693482589933, 'CalciumChannelBlockers_processed': 0.67077607620547286, 'BetaBlockers_processed': 0.73342038663070841, 'Estrogens_processed': 0.91742778917335122, 'ADHD_processed': 0.90444448730766458, 'Statins_processed': 0.82841237909564536, 'Antihistamines_processed': 0.72860533391663285, 'ProtonPumpInhibitors_processed': 0.82000178720794759, 'Opiods_processed': 0.82089058991342667, 'AtypicalAntipsychotics_processed': 0.8793452344413768}


In [2]:
import ast
dic = eval("{'NSAIDS_processed': 0.8878757023735081, 'ACEInhibitors_processed': 0.82898028932750711, 'SkeletalMuscleRelaxants_processed': 0.71413229709440951, 'Triptans_processed': 0.90845402091448257, 'OralHypoglycemics_processed': 0.87597693482589933, 'CalciumChannelBlockers_processed': 0.67077607620547286, 'BetaBlockers_processed': 0.73342038663070841, 'Estrogens_processed': 0.91742778917335122, 'ADHD_processed': 0.90444448730766458, 'Statins_processed': 0.82841237909564536, 'Antihistamines_processed': 0.72860533391663285, 'ProtonPumpInhibitors_processed': 0.82000178720794759, 'Opiods_processed': 0.82089058991342667, 'AtypicalAntipsychotics_processed': 0.8793452344413768}")


In [3]:
import pandas as pd
pd.Series(dic)

ACEInhibitors_processed              0.828980
ADHD_processed                       0.904444
Antihistamines_processed             0.728605
AtypicalAntipsychotics_processed     0.879345
BetaBlockers_processed               0.733420
CalciumChannelBlockers_processed     0.670776
Estrogens_processed                  0.917428
NSAIDS_processed                     0.887876
Opiods_processed                     0.820891
OralHypoglycemics_processed          0.875977
ProtonPumpInhibitors_processed       0.820002
SkeletalMuscleRelaxants_processed    0.714132
Statins_processed                    0.828412
Triptans_processed                   0.908454
dtype: float64